## Importation des modules

In [1]:
from answers_class import LinearRegression
from utils.export_files import ExportFiles
from application.shared import data
import pandas as pd

#### Calcul des nouvelles variables

In [2]:
# Recuperation de la table common_player_info
df = data['common_player_info'].copy()
df.head(5)

,person_id,first_name,last_name,display_first_last,display_last_comma_first,display_fi_last,player_slug,birthdate,school,country,...,playercode,from_year,to_year,dleague_flag,nba_flag,games_played_flag,draft_year,draft_round,draft_number,greatest_75_flag
0,76001,Alaa,Abdelnaby,Alaa Abdelnaby,"Abdelnaby, Alaa",A. Abdelnaby,alaa-abdelnaby,1968-06-24 00:00:00,Duke,USA,...,HISTADD_alaa_abdelnaby,1990.0,1994.0,N,Y,Y,1990,1,25,N
1,76003,Kareem,Abdul-Jabbar,Kareem Abdul-Jabbar,"Abdul-Jabbar, Kareem",K. Abdul-Jabbar,kareem-abdul-jabbar,1947-04-16 00:00:00,UCLA,USA,...,HISTADD_kareem_abdul-jabbar,1969.0,1988.0,N,Y,Y,1969,1,1,Y
2,1505,Tariq,Abdul-Wahad,Tariq Abdul-Wahad,"Abdul-Wahad, Tariq",T. Abdul-Wahad,tariq-abdul-wahad,1974-11-03 00:00:00,San Jose State,France,...,tariq_abdul-wahad,1997.0,2003.0,N,Y,Y,1997,1,11,N
3,949,Shareef,Abdur-Rahim,Shareef Abdur-Rahim,"Abdur-Rahim, Shareef",S. Abdur-Rahim,shareef-abdur-rahim,1976-12-11 00:00:00,California,USA,...,shareef_abdur-rahim,1996.0,2007.0,N,Y,Y,1996,1,3,N
4,76005,Tom,Abernethy,Tom Abernethy,"Abernethy, Tom",T. Abernethy,tom-abernethy,1954-05-06 00:00:00,Indiana,USA,...,HISTADD_tom_abernethy,1976.0,1980.0,N,Y,Y,1976,3,43,N


In [3]:
# instanciation de la classe
exportfiles = ExportFiles()

# initialisation des repertoires d'exportation
tables_dir = '../Exportations/tables'
figures_dir = '../Exportations/images'

# exporter la table au format csv
exportfiles.export_to_csv_format(df, f'{tables_dir}/ml_table1.csv')

# exporter la table au format excel
exportfiles.export_to_xlsx_format(df, f'{tables_dir}/ml_table1.xlsx')

True

In [4]:
# Selection des joueurs qui ne jouent plus en 2022
df = df[df["to_year"] <= 2022]

# recuperation de l'année de naissance
df["birth_year"] = pd.to_numeric(
    df["birthdate"].apply(lambda x: x[:4])
)

# conversion de draft_year en numérique et gestion des erreur
# dans draft_year il y'a une modalité undrafted, donc elle est
# remplacée par NA
df["draft_year"] = pd.to_numeric(
    df["draft_year"], errors="coerce"
)

# remplacer les NA par 0
df["draft_year"] = df["draft_year"].fillna(0)

# Age au moment du draft
df["age_at_draft"] = (
    df["draft_year"] - df["birth_year"]
)

# Selectionner les ages au moment de la draft positifs
# (à cause du traitement en ammont (NA => 0), il peut avoir des ages negatifs)
df = df[df["age_at_draft"] > 0]

In [5]:
# exporter la table ?
# exporter la table au format csv
exportfiles.export_to_csv_format(df, f'{tables_dir}/ml_df_nettoyee.csv')

True

#### Instanciation de la classe

In [6]:
model_hand = LinearRegression(
    df=df,
    y_var=['season_exp'],
    x_vars=['age_at_draft', 'position'],
    fit_intercept=True
)

#### Ajustement du modèle

$\quad$ `perfom_linear_reg` utilise la méthode `fit` que nous avons implémentée dans notre classe `LinearRegression`.

In [7]:
resultats = model_hand.perfom_linear_reg()

In [8]:
poste_mapping = {
    'position_Center-Forward': 'Pivot-Ailier fort',
    'position_Forward': 'Ailier',
    'position_Forward-Center': 'Ailier fort-Pivot',
    'position_Forward-Guard': 'Ailier-Arrière',
    'position_Guard': 'Arrière',
    'position_Guard-Forward': 'Arrière-Ailier'
}

resultats['estimation_results']['Variable'] = resultats['estimation_results']['Variable'].replace(poste_mapping)

In [9]:
resultats['estimation_results']

,Variable,Estimation,Borne inférieure,Borne supérieure
0,intercept,13.080794,11.214000,14.947588
1,age_at_draft,-0.304521,-0.385970,-0.223071
2,Pivot-Ailier fort,4.034691,2.580194,5.489188
3,Ailier,-0.723449,-1.242224,-0.204674
4,Ailier fort-Pivot,3.001186,1.803236,4.199137
5,Ailier-Arrière,1.318791,-0.203041,2.840623
6,Arrière,-0.779851,-1.300982,-0.258720
7,Arrière-Ailier,1.889610,0.805603,2.973616


In [10]:
# exporter les resultats du modèle ?
# exporter la table au format csv
exportfiles.export_to_csv_format(
    resultats['estimation_results'],
    f'{tables_dir}/ml_results.csv'
)

True

#### Resultats avec la class de sci-kit learn

In [11]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [12]:
df.dropna(subset=['season_exp', 'age_at_draft', 'position'], inplace=True)
position_dummies = pd.get_dummies(df.loc[:, 'position'], drop_first=True)
X = pd.concat([df.loc[:, ['age_at_draft']], position_dummies], axis=1)
X = np.asarray(X).reshape((X.shape[0], X.shape[1]))
y = np.asarray(df.loc[:,'season_exp']).reshape(df.shape[0], 1)

In [13]:
model = LinearRegression(fit_intercept=True)
model.fit(X,y)

LinearRegression()

In [14]:
model.coef_

array([[-0.30452051,  4.03469136, -0.72344879,  3.00118641,  1.31879082,
        -0.77985085,  1.88960963]])

In [15]:
model.intercept_

array([13.08079382])

Si on regarde `model.coef_` on se rend compte que les résultats de `scikit-learn` sont les mêmes que ceux obtenu avec notre classe `LinearRegression`. En regardant `model.intercept_` on a les mêmes intercepts.

In [16]:
variables = np.asarray(resultats['estimation_results']['Variable']).\
    reshape((resultats['estimation_results']['Variable'].shape[0], 1))
estimations_hand = np.asarray(resultats['estimation_results']['Estimation']).\
    reshape((resultats['estimation_results']['Estimation'].shape[0], 1))
estimations_sk_learn = np.vstack((
    model.intercept_.reshape((model.intercept_.shape[0],1)),
     model.coef_[0].reshape((model.coef_[0].shape[0], 1))
))

In [17]:
df_comparaison = pd.DataFrame(
    data=np.hstack((variables, estimations_hand, estimations_sk_learn)),
    columns=['Variable', 'Estimation avec notre modele', 'Estimation avec sklearn']
)

In [18]:
df_comparaison

,Variable,Estimation avec notre modele,Estimation avec sklearn
0,intercept,13.080794,13.080794
1,age_at_draft,-0.304521,-0.304521
2,Pivot-Ailier fort,4.034691,4.034691
3,Ailier,-0.723449,-0.723449
4,Ailier fort-Pivot,3.001186,3.001186
5,Ailier-Arrière,1.318791,1.318791
6,Arrière,-0.779851,-0.779851
7,Arrière-Ailier,1.88961,1.88961


In [19]:
# exporter df_comparaison ?
# exporter la table au format csv
exportfiles.export_to_csv_format(
    df_comparaison,
    f'{tables_dir}/ml_comparaison_results_sklearn.csv'
)

True

#### Validation du modèle : K-folds

In [20]:
model_hand.k_fold(5)

[np.float64(4.590710274473485),
 np.float64(4.485591579874982),
 np.float64(4.67588530163391),
 np.float64(4.764901646856069),
 np.float64(4.511956470515589)]

In [21]:
np.mean(model_hand.k_fold(5))

np.float64(4.601432010889611)

Les erreurs quadratiques moyennes sont autour de 4,6 en moyenne.

#### Prédiction de la durée de carrière d'un joeur lamdba

A ce niveau, nous allons juste nous servir de la méthode implémentée dans la classe pour notre application pour pas avoir à réécrire le code.

In [22]:
from logic_for_application import CareerPrediction
from datetime import date

In [23]:
career_prediction = CareerPrediction(data=data["common_player_info"])

In [24]:
birthdate = date(2000, 1, 5)
draft_date = date(2018, 1, 8)
position = "Center"

In [25]:
results = career_prediction.\
    predict_career_duration(
        birthdate=str(birthdate),
        draft_year=str(draft_date),
        position=position,
        coef_df=career_prediction.run_regression()[
            "estimation_results"
        ],
    )

predicted_duration = results["duree_predite"]
lower_bound = results["intervalle_confiance"][0]
upper_bound = results["intervalle_confiance"][1]
print(
    f"Durée de carrière : {predicted_duration:.1f} ans\n"
    f"Intervalle de confiance [{lower_bound:.1f}, {upper_bound:.1f}]"
)

Durée de carrière : 7.6 ans
Intervalle de confiance [4.3, 10.9]
